In [1]:
import pandas as pd
import os
import re

In [2]:
path = '/Users/guangbo_niu/Library/Mobile Documents/com~apple~CloudDocs/Academics/Autumn 2022/DPPP 2/Political_Polarization/data'
rollcalls_data = pd.read_csv(os.path.join(path, 'roll_call/Hall_rollcalls.csv'), low_memory=False)
members_data = pd.read_csv(os.path.join(path, 'roll_call/Hall_members.csv'))
h113_votes = pd.read_csv(os.path.join(path, 'roll_call/H113_votes.csv'))
h114_votes = pd.read_csv(os.path.join(path, 'roll_call/H114_votes.csv'))
h115_votes = pd.read_csv(os.path.join(path, 'roll_call/H115_votes.csv'))

In [3]:
# merge votes, members data, and roll call data
def merge_votes_members_rollcalls(votes, members, roll_calls, congress):
    df = pd.merge(votes, 
                  members[members['congress']==congress].loc[:, ['icpsr', 'party_code']], 
                  on='icpsr')
    df = pd.merge(df, 
                  roll_calls[roll_calls['congress']==congress].loc[:, ['rollnumber', 'date', 'bill_number', 'vote_result', 'vote_desc', 'vote_question']],
                  on='rollnumber')
    #df['rollnumber']=df['congress'].astype(str) + '_' + df['rollnumber'].astype(str)
    return df
    
h113_votes = (merge_votes_members_rollcalls(h113_votes, members_data, rollcalls_data, 113))
h114_votes = (merge_votes_members_rollcalls(h114_votes, members_data, rollcalls_data, 114))
h115_votes = (merge_votes_members_rollcalls(h115_votes, members_data, rollcalls_data, 115))

In [6]:
def tabulate_merge(df1, df2, df3):
    def tab(df):
        return df.groupby(['congress', 'rollnumber', 'date', 'cast_code'])['party_code'].value_counts()
    df1 = tab(df1)
    df2 = tab(df2)
    df3 = tab(df3)
    df_out = pd.DataFrame(pd.concat([df1, df2, df3], axis=0))
    return df_out.rename({'party_code': 'counts'}, axis=1)


votes_counts = tabulate_merge(h113_votes, h114_votes, h115_votes)

In [7]:
votes_counts

counts
congress rollnumber date       cast_code party_code        
113      1          2013-01-03 1         200            220
                               6         100            192
                               7         200             10
                                         100              5
                               9         100              4
...                                                     ...
115      1207       2018-12-21 1         200            152
                               6         200             47
                                         100              1
                               9         200             36
                                         100             33

[19835 rows x 1 columns]